# Машинное обучение в задачах акустической классификации типов дорожных условий для беспилотных автомобилей

Нужно классифицировать типы дорожных условий на основе аудиофайлов формата .wav

In [48]:
import librosa
import soundfile as sf
import os
import pathlib

In [2]:
%%time

genres = 'dry wet static'.split()

for g in genres:
    pathlib.Path(f'splitted/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'./genres/{g}'):
        full_name = f'./genres/{g}/{filename}'
        audio, sr = librosa.load(full_name, sr=None, duration=1200)
        buffer = 1 * sr
        samples_total = len(audio)
        samples_wrote = 0
        counter = 1
        while samples_wrote < samples_total:
            #check if the buffer is not exceeding total samples 
            if buffer > (samples_total - samples_wrote):
                buffer = samples_total - samples_wrote

            block = audio[samples_wrote : (samples_wrote + buffer)]
            out_filename = f'./splitted/{g}/split_' + str(counter) + "_" + f'{filename}'

            # Write 1 second segment
            sf.write(out_filename, block, sr)
            counter += 1
            samples_wrote += buffer

Wall time: 19.2 s


**Данная ячейка кода осуществляет загрузку первых 20 минут аудиофайлов из папки 'genres' и разделяет каждый аудиофайл на равные отрезки по 1 секунде, после чего сохраняет данные отрезки в папку 'splitted'. В дальнейшем буду работать именно с ними**

In [3]:
%%time

header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

Wall time: 0 ns


**В данной ячейке кода я задаю каркас датасета. Задаю названия столбцов**

In [4]:
import pandas as pd
import numpy as np
import csv

In [5]:
%%time

file = open('dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'dry wet static'.split()
for g in genres:
    for filename in os.listdir(f'./splitted/{g}'):
        songname = f'./splitted/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True)
        rmse = librosa.feature.rms(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

C:\Users\vladi\anaconda3\lib\site-packages\librosa\core\pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


Wall time: 3min 15s


**В данной ячейке кода я извлекаю признаки, необходимые мне для обучения моделей машинного обучения, и сохраняю их в csv файл, для удобной работы с ними и дальнейшего преобразования их в dataframe**

In [6]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [7]:
data = pd.read_csv('dataset.csv', on_bad_lines='skip')

In [8]:
data

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,split_1000_dry.wav,0.734281,0.144253,301.040038,781.910873,505.296187,0.007923,-325.962921,192.424301,23.447613,...,13.951578,5.872117,4.919704,3.976071,1.668905,-0.376623,-1.208098,-0.010591,3.580090,dry
1,split_1001_dry.wav,0.763000,0.214444,243.000545,738.519855,328.870739,0.003174,-314.523712,177.839218,31.339277,...,16.749880,7.692034,5.189845,4.168124,1.172801,-0.506250,-0.106552,2.288377,4.761861,dry
2,split_1002_dry.wav,0.699385,0.112530,336.394923,863.757702,611.738725,0.006314,-339.772980,200.919144,10.475849,...,13.418714,6.546124,6.844330,3.474560,1.796749,-1.324297,-0.941631,-0.106851,2.412918,dry
3,split_1003_dry.wav,0.723162,0.136663,286.629039,762.639538,508.721924,0.006869,-340.599335,205.149261,13.462376,...,12.634643,4.674061,5.596658,3.925817,2.197597,-1.428367,0.982987,3.307850,4.154649,dry
4,split_1004_dry.wav,0.742990,0.125296,279.592298,780.810990,443.632923,0.004561,-354.149445,196.656540,25.438204,...,14.922936,4.740580,3.938826,4.482189,3.433761,1.694127,0.235758,1.269113,3.256027,dry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,split_997_static.wav,0.651850,0.011685,474.217087,1459.523003,390.534002,0.006181,-567.838867,147.493912,56.827988,...,11.939364,2.573770,7.555421,1.780822,7.179546,1.305689,6.611869,1.798986,5.279230,static
4796,split_998_static.wav,0.609351,0.015689,409.978448,1353.229245,286.049028,0.005915,-572.092224,144.467102,62.100346,...,12.816906,5.733082,7.072026,3.952815,7.898604,2.423805,5.330413,1.821360,5.035167,static
4797,split_999_static.wav,0.608383,0.010871,508.947161,1525.978092,396.896085,0.006115,-574.067322,139.975357,55.984364,...,13.362040,3.352252,7.953781,1.893215,6.167076,0.306760,6.646135,2.302438,5.128450,static
4798,split_99_static.wav,0.722868,0.007113,686.808347,1860.821824,619.079590,0.006880,-606.802246,114.540108,53.540844,...,14.782586,3.386567,9.030226,1.274534,8.364443,2.395092,9.882740,2.657761,5.966316,static


In [9]:
%%time

# Удаление ненужных столбцов
data = data.drop(['filename'],axis=1)
# Создание меток
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
# Масштабирование столбцов признаков
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
# Разделение данных на обучающий и тестовый набор
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Wall time: 4 ms


**В данной ячейке кода я произвожу удаление ненужного столбца из датафрема ('filename') т.к. он не нужен нам для обучения моделей. После этого создаю метки классов (сухая дорога, мокрая дорога, стоячая машина) и произвожу масштабированеи стобцов признаков, используя StandardScaler для приведения их к нулевому среднему и единичному стандартному отклонению, после чего разбиваю данные на тренировочную и тестовые выборки в соотношении 80/20**

In [10]:
data

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0.734281,0.144253,301.040038,781.910873,505.296187,0.007923,-325.962921,192.424301,23.447613,-1.798386,...,13.951578,5.872117,4.919704,3.976071,1.668905,-0.376623,-1.208098,-0.010591,3.580090,dry
1,0.763000,0.214444,243.000545,738.519855,328.870739,0.003174,-314.523712,177.839218,31.339277,-1.276680,...,16.749880,7.692034,5.189845,4.168124,1.172801,-0.506250,-0.106552,2.288377,4.761861,dry
2,0.699385,0.112530,336.394923,863.757702,611.738725,0.006314,-339.772980,200.919144,10.475849,4.924136,...,13.418714,6.546124,6.844330,3.474560,1.796749,-1.324297,-0.941631,-0.106851,2.412918,dry
3,0.723162,0.136663,286.629039,762.639538,508.721924,0.006869,-340.599335,205.149261,13.462376,1.143430,...,12.634643,4.674061,5.596658,3.925817,2.197597,-1.428367,0.982987,3.307850,4.154649,dry
4,0.742990,0.125296,279.592298,780.810990,443.632923,0.004561,-354.149445,196.656540,25.438204,-0.089451,...,14.922936,4.740580,3.938826,4.482189,3.433761,1.694127,0.235758,1.269113,3.256027,dry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.651850,0.011685,474.217087,1459.523003,390.534002,0.006181,-567.838867,147.493912,56.827988,19.592678,...,11.939364,2.573770,7.555421,1.780822,7.179546,1.305689,6.611869,1.798986,5.279230,static
4796,0.609351,0.015689,409.978448,1353.229245,286.049028,0.005915,-572.092224,144.467102,62.100346,24.292639,...,12.816906,5.733082,7.072026,3.952815,7.898604,2.423805,5.330413,1.821360,5.035167,static
4797,0.608383,0.010871,508.947161,1525.978092,396.896085,0.006115,-574.067322,139.975357,55.984364,25.302305,...,13.362040,3.352252,7.953781,1.893215,6.167076,0.306760,6.646135,2.302438,5.128450,static
4798,0.722868,0.007113,686.808347,1860.821824,619.079590,0.006880,-606.802246,114.540108,53.540844,29.811792,...,14.782586,3.386567,9.030226,1.274534,8.364443,2.395092,9.882740,2.657761,5.966316,static


**Здесь я вывожу полученный dataframe для демонстрации полученных данных и проверки кореектности их заполнения и масштабирования**

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

**Объявляю методы машинного обучения, которые буду использовать для решения поставленной задачи, после чего приступаю к непосредственному их применению**

**Метод опорных векторов (SVM)**

In [12]:
model = SVC(kernel='linear')

In [13]:
%%time

# Метод опорных векторов (SVM)
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
print("SVM accuracy:", accuracy_score(y_test, y_pred_svm))
print("SVM classification report:\n", classification_report(y_test, y_pred_svm))
print("SVM confusion matrix:\n", confusion_matrix(y_test, y_pred_svm))

SVM accuracy: 0.9864583333333333
SVM classification report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.97       224
           1       1.00      0.99      1.00       244
           2       0.98      0.99      0.99       492

    accuracy                           0.99       960
   macro avg       0.99      0.98      0.99       960
weighted avg       0.99      0.99      0.99       960

SVM confusion matrix:
 [[217   0   7]
 [  1 242   1]
 [  4   0 488]]
Wall time: 90.6 ms


In [14]:
%%time

cv_results = cross_val_score(model,                  # модель
                             X,                      # матрица признаков
                             y,                      # вектор цели
                             cv = 5,                 # тип разбиения
                             scoring = 'accuracy',   # метрика
                             n_jobs=-1)              # используются все ядра CPU

print("Кросс-валидация: ", cv_results)
print("Среднее по кросс-валидации: ", cv_results.mean())
print("Дисперсия по кросс-валидации: ", cv_results.std())

Кросс-валидация:  [0.93229167 0.98020833 0.98541667 0.953125   0.90208333]
Среднее по кросс-валидации:  0.9506249999999999
Дисперсия по кросс-валидации:  0.030933115570354205
Wall time: 3.22 s


In [15]:
%%time

param_grid = {'C': [0.1, 1, 10],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
              'gamma': ['scale', 'auto']}

svm = SVC(random_state=42)

grid_search = GridSearchCV(svm, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Wall time: 2.02 s


In [16]:
grid_search.best_score_

0.9924479166666667

In [17]:
grid_search.best_estimator_

SVC(C=10, random_state=42)

**Gaussian Naive Bayes**

In [18]:
model = GaussianNB()

In [19]:
%%time

# Метод наивного Байеса (Naive Bayes)
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print("Naive Bayes accuracy:", accuracy_score(y_test, y_pred_nb))
print("Naive Bayes classification report:\n", classification_report(y_test, y_pred_nb))
print("Naive Bayes confusion matrix:\n", confusion_matrix(y_test, y_pred_nb))

Naive Bayes accuracy: 0.8802083333333334
Naive Bayes classification report:
               precision    recall  f1-score   support

           0       0.68      0.95      0.79       224
           1       0.98      0.98      0.98       244
           2       0.97      0.80      0.88       492

    accuracy                           0.88       960
   macro avg       0.88      0.91      0.88       960
weighted avg       0.91      0.88      0.88       960

Naive Bayes confusion matrix:
 [[212   2  10]
 [  1 240   3]
 [ 97   2 393]]
Wall time: 9.03 ms


In [20]:
%%time

cv_results = cross_val_score(model,                  # модель
                             X,                      # матрица признаков
                             y,                      # вектор цели
                             cv = 5,                 # тип разбиения
                             scoring = 'accuracy',   # метрика
                             n_jobs=-1)              # используются все ядра CPU

print("Кросс-валидация: ", cv_results)
print("Среднее по кросс-валидации: ", cv_results.mean())
print("Дисперсия по кросс-валидации: ", cv_results.std())

Кросс-валидация:  [0.90520833 0.89479167 0.86979167 0.83645833 0.81979167]
Среднее по кросс-валидации:  0.8652083333333334
Дисперсия по кросс-валидации:  0.03281894777919201
Wall time: 20.1 ms


In [21]:
%%time

param_grid = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]}

gnb = GaussianNB()

grid_search = GridSearchCV(gnb, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

{'var_smoothing': 1e-09}
Wall time: 41.6 ms


In [22]:
grid_search.best_score_

0.8716145833333334

In [23]:
grid_search.best_estimator_

GaussianNB()

**K-Nearest Neighbors**

In [24]:
model = KNeighborsClassifier(n_neighbors=2)

In [25]:
%%time

# Метод k-ближайших соседей (KNN)
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
print("KNN accuracy:", accuracy_score(y_test, y_pred_knn))
print("KNN classification report:\n", classification_report(y_test, y_pred_knn))
print("KNN confusion matrix:\n", confusion_matrix(y_test, y_pred_knn))

KNN accuracy: 0.9947916666666666
KNN classification report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       224
           1       1.00      1.00      1.00       244
           2       1.00      0.99      0.99       492

    accuracy                           0.99       960
   macro avg       0.99      1.00      0.99       960
weighted avg       0.99      0.99      0.99       960

KNN confusion matrix:
 [[224   0   0]
 [  0 244   0]
 [  5   0 487]]
Wall time: 225 ms


In [26]:
%%time

cv_results = cross_val_score(model,                  # модель
                             X,                      # матрица признаков
                             y,                      # вектор цели
                             cv = 5,                 # тип разбиения
                             scoring = 'accuracy',   # метрика
                             n_jobs=-1)              # используются все ядра CPU

print("Кросс-валидация: ", cv_results)
print("Среднее по кросс-валидации: ", cv_results.mean())
print("Дисперсия по кросс-валидации: ", cv_results.std())

Кросс-валидация:  [0.91875    0.975      0.984375   0.90208333 0.94375   ]
Среднее по кросс-валидации:  0.9447916666666666
Дисперсия по кросс-валидации:  0.031567828208126356
Wall time: 176 ms


In [27]:
%%time

param_grid = {'n_neighbors': [3, 5, 7],
              'weights': ['uniform', 'distance'],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

{'algorithm': 'auto', 'n_neighbors': 3, 'weights': 'uniform'}
Wall time: 864 ms


In [28]:
grid_search.best_score_

0.9833333333333334

In [29]:
grid_search.best_estimator_

KNeighborsClassifier(n_neighbors=3)

**Decision Tree Classifier**

In [30]:
model = DecisionTreeClassifier(random_state=42)

In [31]:
%%time

# Решающие деревья (Decision Trees)
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
print("Decision Trees accuracy:", accuracy_score(y_test, y_pred_dt))
print("Decision Trees classification report:\n", classification_report(y_test, y_pred_dt))
print("Decision Trees confusion matrix:\n", confusion_matrix(y_test, y_pred_dt))

Decision Trees accuracy: 0.9677083333333333
Decision Trees classification report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94       224
           1       0.99      0.98      0.99       244
           2       0.97      0.98      0.97       492

    accuracy                           0.97       960
   macro avg       0.97      0.96      0.97       960
weighted avg       0.97      0.97      0.97       960

Decision Trees confusion matrix:
 [[210   0  14]
 [  4 239   1]
 [ 10   2 480]]
Wall time: 101 ms


In [32]:
%%time

cv_results = cross_val_score(model,                  # модель
                             X,                      # матрица признаков
                             y,                      # вектор цели
                             cv = 4,                 # тип разбиения
                             scoring = 'accuracy',   # метрика
                             n_jobs=-1)              # используются все ядра CPU

print("Кросс-валидация: ", cv_results)
print("Среднее по кросс-валидации: ", cv_results.mean())
print("Дисперсия по кросс-валидации: ", cv_results.std())

Кросс-валидация:  [0.93083333 0.97916667 0.9625     0.87916667]
Среднее по кросс-валидации:  0.9379166666666667
Дисперсия по кросс-валидации:  0.03810393315948135
Wall time: 107 ms


In [33]:
%%time

param_grid = {'criterion': ['gini', 'entropy'],
              'max_depth': [5, 10, 15, 20],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4]}

dtc = DecisionTreeClassifier(random_state=42)

grid_search = GridSearchCV(dtc, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

{'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5}
Wall time: 2.28 s


In [34]:
grid_search.best_score_

0.9619791666666666

In [35]:
grid_search.best_estimator_

DecisionTreeClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=5,
                       random_state=42)

**Случайный лес**

In [36]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [37]:
%%time

# Случайный лес (Random Forest)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Random Forest accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest classification report:\n", classification_report(y_test, y_pred_rf))
print("Random Forest confusion matrix:\n", confusion_matrix(y_test, y_pred_rf))

Random Forest accuracy: 0.984375
Random Forest classification report:
               precision    recall  f1-score   support

           0       0.97      0.96      0.97       224
           1       1.00      1.00      1.00       244
           2       0.98      0.99      0.99       492

    accuracy                           0.98       960
   macro avg       0.98      0.98      0.98       960
weighted avg       0.98      0.98      0.98       960

Random Forest confusion matrix:
 [[215   0   9]
 [  1 243   0]
 [  5   0 487]]
Wall time: 1.77 s


In [38]:
%%time

cv_results = cross_val_score(model,                  # модель
                             X,                      # матрица признаков
                             y,                      # вектор цели
                             cv = 5,                 # тип разбиения
                             scoring = 'accuracy',   # метрика
                             n_jobs=-1)              # используются все ядра CPU

print("Кросс-валидация: ", cv_results)
print("Среднее по кросс-валидации: ", cv_results.mean())
print("Дисперсия по кросс-валидации: ", cv_results.std())

Кросс-валидация:  [0.925      0.98229167 0.99270833 0.940625   0.90520833]
Среднее по кросс-валидации:  0.9491666666666667
Дисперсия по кросс-валидации:  0.03341396497806801
Wall time: 1.9 s


In [39]:
%%time

param_grid = {'n_estimators': [50, 100, 200],
              'max_depth': [10, 20, None],
              'max_features': ['sqrt', 'log2', None]}

rf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

{'max_depth': 20, 'max_features': 'log2', 'n_estimators': 100}
Wall time: 46.9 s


In [40]:
grid_search.best_score_

0.9841145833333333

In [41]:
grid_search.best_estimator_

RandomForestClassifier(max_depth=20, max_features='log2', random_state=42)

**Gradient Boosting Classifier**

In [42]:
model = GradientBoostingClassifier()

In [43]:
%%time

#  Gradient Boosting Classifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
y_pred_gbc = gbc.predict(X_test)
print("Gradient Boosting Classifier accuracy:", accuracy_score(y_test, y_pred_gbc))
print("Gradient Boosting Classifier classification report:\n", classification_report(y_test, y_pred_gbc))
print("Gradient Boosting Classifier confusion matrix:\n", confusion_matrix(y_test, y_pred_gbc))

Gradient Boosting Classifier accuracy: 0.984375
Gradient Boosting Classifier classification report:
               precision    recall  f1-score   support

           0       0.98      0.96      0.97       224
           1       1.00      1.00      1.00       244
           2       0.98      0.99      0.99       492

    accuracy                           0.98       960
   macro avg       0.99      0.98      0.98       960
weighted avg       0.98      0.98      0.98       960

Gradient Boosting Classifier confusion matrix:
 [[214   0  10]
 [  1 243   0]
 [  4   0 488]]
Wall time: 14.8 s


In [44]:
%%time

cv_results = cross_val_score(model,                  # модель
                             X,                      # матрица признаков
                             y,                      # вектор цели
                             cv = 5,                 # тип разбиения
                             scoring = 'accuracy',   # метрика
                             n_jobs=-1)              # используются все ядра CPU

print("Кросс-валидация: ", cv_results)
print("Среднее по кросс-валидации: ", cv_results.mean())
print("Дисперсия по кросс-валидации: ", cv_results.std())

Кросс-валидация:  [0.925      0.97604167 0.99270833 0.94791667 0.903125  ]
Среднее по кросс-валидации:  0.9489583333333332
Дисперсия по кросс-валидации:  0.03262928345656535
Wall time: 15.4 s


In [45]:
%%time

param_grid = {'learning_rate': [0.1, 0.05, 0.01],
              'n_estimators': [50, 100, 200],
              'max_depth': [3, 5, 7]}

gb = GradientBoostingClassifier(random_state=42)

grid_search = GridSearchCV(gb, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

{'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200}
Wall time: 5min 28s


In [46]:
grid_search.best_score_

0.9838541666666666

In [47]:
grid_search.best_estimator_

GradientBoostingClassifier(max_depth=7, n_estimators=200, random_state=42)

**В результате обучения и тестирования полученных моделей получаю очень высокие показатели accuracy, что свидетельствует о точности моделей, что может быть связано с недостатком различных классов. Также получаю высокую точность с применением кроссвалидации с разбиением на 5 фолдов. После кросс валидации применяю к каждой модели gridsearch, чтобы добиться наилучших результатов с наилучшими параметрами. Если получить достаточное кол-во данных для создания еще нескольких классов, то точность модели может уменьшиться**